In [1]:
from motion.BVH import BVH as BVH
from net.quat import quat
from net.euler import euler
import numpy as np
import re
import copy
import math
filename = "./data/Horse/02/run.bvh"
filename2 = "./data/Human/016/16_55.bvh"

horse_skeleton = BVH.load(filename)     
human_skeleton = BVH.load(filename2)

In [2]:
def scale_centaur(node, num):
    for child in node:
        child.offsets = [x * num for x in child.offsets]
        scale_centaur(child.children, num) 

In [3]:
centaur_skeleton = copy.deepcopy(human_skeleton)
centaur_skeleton.root = copy.deepcopy(horse_skeleton.root)
centaur_skeleton.root.children[0].children[0].children[0].children[0].children[0].children[0] = copy.deepcopy(human_skeleton.root.children[2])
scale_centaur(centaur_skeleton.root.children[0].children[0].children[0].children[0].children[0].children[0].children, 9)

In [4]:
human_skeleton.calculate_global_positions(human_skeleton.root)
horse_skeleton.calculate_global_positions(horse_skeleton.root)
centaur_skeleton.calculate_global_positions(centaur_skeleton.root)

In [5]:
def extract_hierarchy(bvh_file, skeleton):
    for child in skeleton:
        if "End" in child.name:
            bvh_file.write("End Site\n")
            bvh_file.write("{\n")
            bvh_file.write(f"OFFSET {float(child.offsets[0])} {float(child.offsets[1])} {float(child.offsets[2])}\n")
            bvh_file.write("}\n")
            return
        else:
            bvh_file.write(f"JOINT {child.name}\n")
            bvh_file.write("{\n")
            bvh_file.write(f"OFFSET {float(child.offsets[0])} {float(child.offsets[1])} {float(child.offsets[2])}\n")
            bvh_file.write(f"CHANNELS {len(child.channels)} {child.channels[0]} {child.channels[1]} {child.channels[2]}\n")
            extract_hierarchy(bvh_file, child.children)
            bvh_file.write("}\n")

In [6]:
def extract_motiona(skeleton, frame, frame_data, root = True):
    prevframe = frame
    position = None
    
    if(frame >= len(skeleton.rotations)):
        frame = frame % len(skeleton.rotations)

    if root:
        # Get the Xpos, Ypos, Zpos
        position = [skeleton.positions[frame][0],skeleton.positions[frame][1],skeleton.positions[frame][2]]

        # Get the X, Y, Z rotations
        rotation = [skeleton.euler_rotation[frame][0],skeleton.euler_rotation[frame][1],skeleton.euler_rotation[frame][2]]

        #rotation = skeleton.euler_rotation[frame]
        frame_data.append([f'{x:f}' for x in position])
        frame_data.append([f'{x:f}' for x in rotation])
    else:
        rotation = skeleton.euler_rotation[frame]
        frame_data.append([f'{x:f}' for x in rotation])
    
    for child in skeleton.children:
        if("End" in child.name):
            break
        extract_motion(child, prevframe, frame_data, False)



In [7]:
def extract_motion(skeleton, frame, nframes, fps, frame_data ,root = True):
    prevframe = frame
    position = None
    
    if(len(skeleton.rotations) < nframes):
        frame = math.floor(frame % math.ceil(nframes/len(skeleton.rotations))*fps) % len(skeleton.rotations)

    if root:
        # Get the Xpos, Ypos, Zpos
        position = [skeleton.positions[frame][0],skeleton.positions[frame][1],skeleton.positions[frame][2]]

        # Get the X, Y, Z rotations
        rotation = [skeleton.euler_rotation[frame][0],skeleton.euler_rotation[frame][1],skeleton.euler_rotation[frame][2]]

        #rotation = skeleton.euler_rotation[frame]
        frame_data.append([f'{x:f}' for x in position])
        frame_data.append([f'{x:f}' for x in rotation])
    else:
        rotation = skeleton.euler_rotation[frame]
        frame_data.append([f'{x:f}' for x in rotation])

    for child in skeleton.children:
        if("End" in child.name):
            break
        extract_motion(child, prevframe, nframes, fps, frame_data, False)




In [8]:
def write_bvh_file(filename, skeleton):
    with open(filename, 'w') as bvh_file:
        # Write BVH header
        bvh_file.write("HIERARCHY\n")
        bvh_file.write("ROOT " + skeleton.root.name + "\n")
        bvh_file.write("{\n")

        bvh_file.write(f"OFFSET {float(skeleton.root.offsets[0])} {float(skeleton.root.offsets[1])} {float(skeleton.root.offsets[2])} \n")
        bvh_file.write(f"CHANNELS {len(skeleton.root.channels)} {skeleton.root.channels[0]} {skeleton.root.channels[1]} {skeleton.root.channels[2]} {skeleton.root.channels[3]} {skeleton.root.channels[4]} {skeleton.root.channels[5]}\n")
        
        # Write joint hierarchy
        extract_hierarchy(bvh_file, skeleton.root.children)
        bvh_file.write("}\n")
        
        # Write motion data
        bvh_file.write("MOTION\n")
        bvh_file.write(f"Frames: {int(skeleton.nframes)}\n")
        bvh_file.write(f"Frame Time: {float(skeleton.frame_time)}\n")
        
        frame_data = []

        for frame in range(skeleton.nframes):
            extract_motion(skeleton.root, frame, skeleton.nframes, skeleton.frame_time ,frame_data)
            frame_data = re.sub('[\[\],\']', '', str(frame_data))
            bvh_file.write(f"{str(frame_data)}\n")
            frame_data = []

       



In [9]:
write_bvh_file("test.bvh", centaur_skeleton)


In [10]:
human_skeleton.root.positions_local

[array([  0.3723,  16.767 , -36.3147]),
 array([-3.30953134, 34.19905269, 20.48170232]),
 array([15.46173908, 36.71576908, -1.83048489]),
 array([  7.9315225 ,  35.90181452, -15.17652287]),
 array([  4.34875423,  37.84054971, -11.09117418]),
 array([ -2.68162532,  36.95947691, -13.86911316]),
 array([-14.27762014,  29.42004333, -22.09713178]),
 array([-19.16789441,  22.82150871, -25.73435899]),
 array([-29.242469  ,  18.70991888, -18.340183  ]),
 array([-31.73560684,  -3.95554085, -22.55283891]),
 array([ 17.65484569,   4.02844825, -34.55801191]),
 array([-22.48877819, -23.56005922, -21.20351486]),
 array([ 21.62156031, -20.50678702, -24.75981546]),
 array([ 32.90321896,  -8.75005052, -18.23200899]),
 array([ 32.40617799,  13.59858891, -15.71445079]),
 array([ 32.67689136,  12.87063572, -15.44065735]),
 array([ 34.84476976,   6.6007162 , -14.26728929]),
 array([ 26.46103568,  24.71086778, -11.8422913 ]),
 array([14.24985048, 35.01930604, -3.24222635]),
 array([20.26312588, 30.55768044,